In [1]:
import pandas as pd
import numpy as np
import math

ALPHA = 0.6
eps = 1e-4
full_data_1 = pd.read_csv("/Users/hasan.nayir/Projects/Payten/app2scale_reinforcement_learning/server_client_v2/output_new_1.csv")
full_data_2 = pd.read_csv("/Users/hasan.nayir/Projects/Payten/app2scale_reinforcement_learning/server_client_v2/output_new_1_test_deployment.csv")
data = pd.concat([full_data_1, full_data_2]).reset_index()

In [6]:
data["utilization"] = 0.5*np.minimum(data["cpu_usage"]/(data["cpu"]/10),1)+ 0.5*np.minimum(data["memory_usage"]/(data["heap"]/10),1)
# data["performance"] = (1-np.exp(-data["utilization"]*(1-(data["response_time"])/(60))))/(1-data["utilization"]+eps)
data["performance"] = (1/data["response_time"])/40
data["performance"] = (data["performance"]-data["performance"].min())/(data["performance"].max()-data["performance"].min())
data["reward"] = data["performance"] * ALPHA + data["utilization"] * (1-ALPHA)

# for i in range(0, data.shape[0]):
#     temp = data.iloc[i,:]
#     if temp["response_time"] >= 60:
#         data.iloc[i, data.columns.get_loc("reward")] = max((1-np.exp(-0.8*(1-(temp["response_time"])/(60))))/(1-0.8),-10)
#     else:
#         data.iloc[i, data.columns.get_loc("reward")] = (1-np.exp(-0.8))/(1-0.8)



# exp hesap olsa da 0 ile 1 arasına düşürmek lazım
# exp olmasına gerek yok.

In [7]:
data.to_csv("response_time_new_data.csv", index=False)

In [16]:
data["utilization"] = 0.5*np.minimum(data["cpu_usage"]/(data["cpu"]/10),1)+ 0.5*np.minimum(data["memory_usage"]/(data["heap"]/10),1)

data["reward"] = ALPHA*(1-(1/(1+np.exp(-0.5*(data["response_time"]-60))))) + (1-ALPHA)*(1-(1/(1+np.exp(0.5*(data["utilization"]-0.8)))))
# data["reward"] = 1/(1+np.exp(-data["reward"]))

In [17]:
data.to_csv("resp_reward_data.csv", index=False)

In [40]:
## This will consider action cost
# performance > cost
# cpu > replica
cost_per_replica = 0.4
cost_per_heap = 0.4
cost_per_cpu = 0.2
action_cost_list = []

for i in range(0, data.shape[0]):
    temp_instant = data.iloc[i,:]
    
    if i>0:
        temp_previous = data.iloc[i-1,:]
        replica_difference = abs(temp_previous["replica"] - temp_instant["replica"])/(data["replica"].max()-data["replica"].min())
        cpu_limit_difference = abs(temp_previous["cpu"] - temp_instant["cpu"])/(data["cpu"].max()-data["cpu"].min())
        heap_limit_difference = abs(temp_previous["heap"] - temp_instant["heap"])/(data["heap"].max()-data["heap"].min())

        replica_cost = replica_difference * cost_per_replica
        cpu_limit_cost = cpu_limit_difference * cost_per_cpu
        heap_limit_cost = heap_limit_difference * cost_per_heap

    else:
        replica_cost = 0
        cpu_limit_cost = 0
        heap_limit_cost = 0
    

    action_cost_list.append(replica_cost+cpu_limit_cost+heap_limit_cost)

data["action_cost"] = action_cost_list
data["utilization"] = 0.5*np.minimum(data["cpu_usage"]/(data["cpu"]/10),1)+ 0.5*np.minimum(data["memory_usage"]/(data["heap"]/10),1)
data["performance"] = data["num_request"]/data["expected_tps"]

performance_weight = 0.5
utilization_weight = 0.3
action_cost_weight = 0.2

data["reward"] = data["performance"] * performance_weight + data["utilization"] * utilization_weight - data["action_cost"] * action_cost_weight




In [36]:
data.to_csv("action_reward_data.csv", index=False)